In [1]:
import mitsuba as mi
import drjit as dr
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random 
import imageio
import cv2
import numpy as np

In [6]:
scene_path = os.path.join('', 'scenes') 
results_spatio_path = os.path.join('','results_spatio') 
mi.set_variant('scalar_rgb')
texture_path = os.path.join(scene_path,'textures')

tokki = os.path.join(scene_path,'meshes/bunny.ply')


In [11]:
cam1 = mi.load_dict({
'type': 'perspective',
'fov': 45, #24.6 mm
'to_world': mi.ScalarTransform4f.look_at(
    origin=[0.03, 0, -0.7],  # increase the z-coordinate to move the camera farther away
    target=[0.03, 1, -15],
    up=[0, 0, 1]
),
'film2': {
    'type': 'hdrfilm',
    'width': 360,
    'height': 360,
}
# 'distortion': [0, 0, 0, 0, 0]
})

In [12]:
import cgi, os
files = os.listdir('dir/graycode_pattern/')

for f in files:
    s = "dir/graycode_pattern/"
    s = s + f
    scene1 = mi.load_dict({
        'type':'scene',
        'integrator': {'type': 'direct'},
        'light': {
            'type': 'projector',
            'irradiance': {
                'type': 'bitmap',
                'filename': s, 
            },
            'fov': 45,
            'to_world': mi.ScalarTransform4f.look_at(
                origin=[-0.03, 0.1, -0.7],
                target=[-0.03, -0.1, -15],
                up=[0, 0, 1]
            )
            },
        'tok':{
            'type': 'ply',
            'filename': tokki,
            'to_world': mi.ScalarTransform4f.translate([0, 0, -1]),
            'bsdf': {
                'type': 'diffuse',
                'reflectance': {'type': 'rgb', 'value': [0.1, 0.2, 0.3]},
            }
        }
    })

    image = mi.render(scene=scene1, sensor=cam1, spp=64)
    image = np.clip(image * 1000, 0, 1000).astype(np.uint8)
    st = "./results/resultfar_" + f
    # Save the image as a PNG file using imageio
    imageio.imwrite(st, image)


In [8]:
import numpy as np
from math import tan, radians

# Define the field of view in degrees
FOV = 45.0

# Define the focal length in mm
focal_length = 50.0

# Define the image resolution
image_size = (360, 360)  # width, height in pixels

# Calculate the sensor size from the FOV and focal length
sensor_size = (image_size[0] / 2.0) / tan(radians(FOV) / 2.0), (image_size[1] / 2.0) / tan(radians(FOV) / 2.0)

# Calculate the horizontal and vertical pixel sizes
pixel_size = (sensor_size[0] / image_size[0], sensor_size[1] / image_size[1])

# Calculate the scaling factors for the focal length
fx = focal_length / pixel_size[0]
fy = focal_length / pixel_size[1]

# Define the principal point at the center of the image
cx = image_size[0] / 2.0
cy = image_size[1] / 2.0

# Construct the camera intrinsic matrix
cam_int = np.array([[fx, 0.0, cx],
                    [0.0, fy, cy],
                    [0.0, 0.0, 1.0]])
f_x = 50
f_y = 50
c_x = 180
c_y = 180

cam_int = np.array([[f_x, 0, c_x],
              [0, f_y, c_y],
              [0, 0, 1]])
cam_int

array([[ 50,   0, 180],
       [  0,  50, 180],
       [  0,   0,   1]])

In [3]:
proj_mat = np.hstack((cam_int, np.zeros((3,1))))
proj_mat

array([[ 41.42135624,   0.        , 180.        ,   0.        ],
       [  0.        ,  41.42135624, 180.        ,   0.        ],
       [  0.        ,   0.        ,   1.        ,   0.        ]])

In [4]:
proj_int = np.array([[360/(2*np.tan(45*np.pi/360)) , 0, 360/2],
                     [0, 360/(2*np.tan(45*np.pi/360)), 360/2],
                     [0, 0, 1]])
proj_int

array([[434.55844123,   0.        , 180.        ],
       [  0.        , 434.55844123, 180.        ],
       [  0.        ,   0.        ,   1.        ]])

In [7]:
# Get the rotation matrix from the camera transformation matrix
cam_transform = mi.ScalarTransform4f.look_at(
    origin=[0.03, 0, -0.7],
    target=[0.03, 1, -15],
    up=[0, 0, 1]
)
cam_to_world = np.array(cam_transform.matrix)
cam_rot_vec, _ = cv2.Rodrigues(cam_to_world[:3, :3])
cam_rot_mat = np.array(cam_rot_vec)

# Define the proj_dist and proj_int matrices
proj_dist = np.zeros((4, 1))

proj =  mi.ScalarTransform4f.look_at(
            origin=[-0.03, 0.1, -0.7],
            target=[-0.03, -0.1, -15],
            up=[0, 0, 1]
        )

proj_int = np.array([[1036.4638, 0, 179.5], [0, 1036.4638, 179.5], [0, 0, 1]])
# Compute the cam_proj_rmat matrix
# cam_proj_rmat = np.dot(proj_int, np.hstack((cam_rot_mat, -np.dot(cam_rot_mat, cam_to_world[:3, 3].reshape(-1,1)))))


cam_proj_rmat = np.dot(proj_int, cam_to_world[:3, :3])
cam_proj_rmat = np.dot(proj_int, cam_int)

cam_proj_rmat

##50mm
##snesor sensor_size = (2 * 50mm * tan(45/2 degrees))





array([[4.29317363e+04, 0.00000000e+00, 1.86742984e+05],
       [0.00000000e+00, 4.29317363e+04, 1.86742984e+05],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [17]:
u = 360
v = 360
point = np.array([u, v, 1])
cam_int = np.array([[275.8, 0, 180], [0, 275.8, 180], [0, 0, 1]], dtype=np.float32)

cam_proj_tvec = -np.linalg.inv(cam_proj_rmat) @ np.linalg.inv(cam_int) @ proj_int @ point
cam_proj_tvec

array([ 4.41898352,  4.41898352, -1.        ])

In [18]:
# mitsuba nmap, dmap
# cam, proj 같은 높이, 같은 방향
# cmap을 통해 확인